In [121]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz


from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
import os
import csv

df1 = pd.read_csv('2014 excel.csv')
df2 = pd.read_csv('2015 excel.csv')
df3 = pd.read_csv('2016 excel.csv')

In [122]:
#merging files#
frames = [df1, df2, df3]
result = pd.concat(frames, axis=0)

In [123]:
#deleting columns and data#
result= result.drop(['Grid Ref: Northing','Grid Ref: Easting','Reference Number','Expr1','Accident Date','Lighting Conditions','1st Road Class'],axis=1)
result= result.dropna(axis=0)

In [124]:

#listing objects#
list1=['Monday','Tuesday','Wednesday','Thursday']
list2=['Friday','Saturday','Sunday']
list3=['Fine without high winds','Fine with high winds']
list4=['Snowing without high winds','Snowing with high winds']
list5=['Raining without high winds','Raining with high winds']
list6=['Fog or mist']
list7=['Minibus (8 to 16 passenger seats)','Bus or coach (17 or more passenger seats)']
list8=['Goods vehicle 3.5 tonnes mgw and under','Goods vehicle over 3.5 tonnes and under 7.5 tonnes mgw','goods','Goods vehicle over 3.5 tonnes and under 7.5 tonnes mgw']
list9=['Motorcycle over 50cc and up to 125cc','Motorcycle over 125cc and up to 500cc','M cycle 50cc and under','Motorcycle over 500cc','Motorcycle Unknown CC']
list10=['Car','Taxi or Private hire car']



In [125]:
#listing day
#1= weekday
#2= weekend
def b(row):
    if row['Day']in list1:
        val=1
    elif row['Day']in list2:
        val=2
    return val
result['Day'] = result.apply(b, axis=1)


#listing weather conditions#
#1= fine
#2= snowing
#3= raining
#4= fog
#5= other
def e(row):
    if row['Weather Conditions']in list3:
        val=1
    elif row['Weather Conditions']in list4:
        val=2
    elif row['Weather Conditions']in list5:
        val=3
    elif row['Weather Conditions']in list6:
        val=4
    else: 
        val=5   
    return val
result['Weather Conditions'] = result.apply(e, axis=1)

#listing type of vehicle

#1= bus
#2= goods vehicle
#3= motorcycle
#4= car
#5= others
def j(row):
    if row['Type of Vehicle']in list7:
        val=1
    elif row['Type of Vehicle']in list8:
        val=2
    elif row['Type of Vehicle']in list9:
        val=3
    elif row['Type of Vehicle']in list10:
        val=4
    else: 
        val=5  
    return val
result['Type of Vehicle'] = result.apply(j, axis=1)


#listing Time
#1= Day-time
#2= Night-time
def c(row):
    if 600< row['Time (24hr)'] <= 1801:
        val=1
    else:
        val=2
    return val
result['Time (24hr)'] = result.apply(c, axis=1)


  
#Sex Casualty dummies
#1= Female
#0= Male

#Casualty Class
#1=Passenger
#2=Driver
#3=Pedestrian


#Casualty Severity
#Fatal=1
#Serious=2
#Slight=3

#Road Surface
#1=Dry
#2=Wet or Damp
#3=Frost or Ice
#4=Snow

result['Sex of Casualty']=np.where(result['Sex of Casualty']=='Female',1,0)
result['Casualty Class']=np.where(result['Casualty Class']=='Passenger',1,np.where(result['Casualty Class']=='Driver',2,3))

result['Road Surface']=np.where(result['Road Surface']=='Dry',1,np.where(result['Road Surface']=='Wet or Damp',2,np.where(result['Road Surface']=='Frost or Ice',3,4)))
                                




In [126]:
print(result)

      Number of Vehicles  Day  Time (24hr)  Road Surface  Weather Conditions  \
0                      1    2            2             1                   1   
1                      1    2            2             1                   1   
2                      1    2            2             1                   1   
3                      1    2            2             2                   1   
4                      2    1            2             2                   1   
5                      2    1            2             2                   1   
6                      2    1            2             2                   1   
7                      2    2            1             2                   5   
8                      1    2            2             4                   2   
9                      1    1            2             1                   1   
10                     2    1            2             1                   1   
11                     2    1           

In [127]:

#dividing into train and test datasets

from sklearn.model_selection import train_test_split
  
X = result.drop('Casualty Severity', axis=1)
y = result['Casualty Severity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)

In [128]:
import matplotlib.pyplot as plt  
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier 

In [129]:
from sklearn.tree import DecisionTreeClassifier  
tree = DecisionTreeClassifier(criterion='gini',max_depth=4,random_state=1)  
tree.fit(X_train, y_train) 
y_pred=tree.predict(X_test)




In [130]:
score = accuracy_score(y_test, y_pred)
print('The accuracy score of decision tree is %.6f' % score)

importance = tree.feature_importances_
column_names = result.columns.values.tolist()
importances = dict(zip(importance, column_names))

important_features = []

for ii in range(len(importance)):
    if ii < 2:
        important_features.append(importances[sorted(importance, reverse = True)[ii]])
        
print('The 3 most important features in decision tree model are: '+str(important_features))


The accuracy score of decision tree is 0.854745
The 3 most important features in decision tree model are: ['Age of Casualty', 'Number of Vehicles']


In [131]:
# Plotting decision tree
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz

dot_data = export_graphviz(tree,
                            filled=True, 
                            rounded=True,
                            out_file=None) 
 
graph.write_png('tree.png')

from IPython.display import Image
Image('tree.png', width=1000)

ModuleNotFoundError: No module named 'pydotplus'